Compute the rolling estimates. The electricity readings of each home are used and a sliding window approach is applied to compute the average energy use estimate.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0, '../src')

import pickle
import datetime

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.ticker as plticker

from datetime import timedelta
from pathlib import Path
from multiprocessing import Pool
from functools import partial

from IdealDataInterface import IdealDataInterface

from config import TIME_FORMAT
from config import SENSOR_DATA_FOLDER, CACHE_FOLDER, CPU_HIGH_MEMMORY, CPU_LOW_MEMMORY
from config import EVALUATION_PERIOD, FFILL_LIMIT, SAMPLING_WINDOW, N_SAMPLES_ROLLING
from config import ROLLING_WINDOW_WIDTH, ROLLING_WINDOW_STEP, SAMPLING_WINDOW_ROLLING, ROLLING_START_DATE
from config import SAMPLING_MASK, IGNORE_HOMES

from utils import treatment_control, load_mains

from sampling import data_to_sample_array, sample_energy, compute_sample_sizes, compute_estimate

In [4]:
# Run plotting styles
%run -i '../src/sns_styles.py'

cmap = sns.color_palette()

In [5]:
df_group = treatment_control()

treatment_homes = df_group.loc[df_group['group'] == 'treatment', 'homeid']
control_homes = df_group.loc[df_group['group'] == 'control', 'homeid']

print('Found {} treatment and {} control homes.'.format(len(treatment_homes), len(control_homes)))

Found 106 treatment and 107 control homes.


In [6]:
# This will create indices to the rolling window slices
def rolling_window_indices(total_length, window_length, window_step):
    """ Adapted from here: https://gist.github.com/alexlouden/e42f1d96982f7f005e62ebb737dcd987 
    Note that in the referenced implementation the last window is not correctly specified. """
    # The "trick" is to create an array which holds the indices of each window in its rows.
    # That is, each row is one "window". Vertically (along the rows) the first column will contain
    # the first index of each respective window. The columns will then just need to "count" up
    # to include the number of elements in the windows.
    
    # The start indices of each window, will be layed out across the rows
    vert_idx_list = np.arange(0, total_length - window_length + 1, window_step)
    
    # Simply add the count for the number of elements in the window.
    # This will be added to each row, i.e. marking the start and the subsequent
    # indices along the columns.
    hori_idx_list = np.arange(window_length)
    
    # Combine the above to create the array
    A, B = np.meshgrid(hori_idx_list, vert_idx_list)
    idx_array = A + B
    
    return idx_array

In [7]:
def estimate_rolling(homeid, ROLLING_START_DATE, ROLLING_WINDOW_WIDTH, ROLLING_WINDOW_STEP, 
                     N_SAMPLES_ROLLING, SAMPLING_MASK, SAMPLING_WINDOW_ROLLING):
    # Load the data
    ts = load_mains(homeid)
    
    # Limit to the requested date range
    ts = ts[ts.index >= ROLLING_START_DATE]
    
    # Divide the data into chunks of rolling windows
    idx_array = rolling_window_indices(ts.shape[0], ROLLING_WINDOW_WIDTH, ROLLING_WINDOW_STEP)
    
    # Create the rolling windows. This will actually slice the time series and store each
    # window in the list. This list is then passed to the multiprocessing.pool instance
    # to compute the estimate for each window
    windows = [ ts.iloc[idx_array[i,:]] for i in range(idx_array.shape[0]) ]
    
    # Compute the estimate per home using all cores
    func = partial(compute_estimate, N=N_SAMPLES_ROLLING, SAMPLING_MASK=SAMPLING_MASK, SAMPLING_WINDOW=SAMPLING_WINDOW_ROLLING)

    with Pool(processes=CPU_LOW_MEMMORY) as pool:
        rolling_estimates = pool.map(func, windows)
        
    # Put everything into a DataFrame
    df_result = pd.DataFrame(rolling_estimates)
    
    # The timestamp will be the left-most timestamp, i.e. the beginning of each roling window
    df_result['time'] = ts.index[idx_array[:,0]]
    
    df_result['homeid'] = homeid

    return df_result

In [8]:
homeids = list(treatment_homes) + list(control_homes)

homeids = set(homeids) - set(IGNORE_HOMES.keys())

fpath = Path('../data/rolling_electricity_estimates/')
if not fpath.is_dir():
    fpath.mkdir()

fname = lambda h: fpath / Path('homeid{}_estimated_rolling_electricty_use.csv'.format(h))

print('Will be skipping compution for homes already found in {}'.format(fpath))

for homeid in homeids:
    # Compute the estimates but skip files that are already present
    if fname(homeid).is_file():
        continue
    try:
        df_estimates = estimate_rolling(homeid, ROLLING_START_DATE, ROLLING_WINDOW_WIDTH, ROLLING_WINDOW_STEP, 
                                        N_SAMPLES_ROLLING, SAMPLING_MASK, SAMPLING_WINDOW_ROLLING)
    except:
        print('Error in home {}.'.format(homeid))
        continue

    # Store to disk
    df_estimates.to_csv(fname(homeid), sep='\t', float_format='%.3f', date_format=TIME_FORMAT, index=False)

Will be skipping compution for homes already found in ../data/rolling_electricity_estimates
